In [1]:
from __future__ import print_function, division
import unittest
from os.path import join
from os import remove
import pandas as pd
from nilmtk.datastore import HDFDataStore
from nilmtk import DataSet
from nilmtk.disaggregate import CombinatorialOptimisation

In [3]:
filename = join("../../data/", 'co_test.h5')
dataset = DataSet(filename)

In [6]:
elec = dataset.buildings[1].elec
co = CombinatorialOptimisation()
co.train(elec)
mains = elec.mains()
pred = co.disaggregate_chunk(mains.load(sample_period=1).next())

Training model for submeter 'ElecMeter(instance=2, building=1, dataset=None, appliances=[])'
Training model for submeter 'ElecMeter(instance=3, building=1, dataset=None, appliances=[])'
Done training!
Estimating power demand for 'ElecMeter(instance=2, building=1, dataset=None, appliances=[])'
Estimating power demand for 'ElecMeter(instance=3, building=1, dataset=None, appliances=[])'


/Users/nipunbatra/git/nilmtk/nilmtk/utils.py:390: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).ffill(limit=15)
  data = data.resample(**resample_kwargs)


In [19]:
gt = {}
for meter in elec.submeters().meters:
    gt[meter] = meter.load(sample_period=1).next().squeeze()

In [22]:
gt = pd.DataFrame(gt)

In [23]:
gt.equals(pred)

False

In [24]:
gt.head()

,"ElecMeter(instance=2, building=1, dataset=None, appliances=[])","ElecMeter(instance=3, building=1, dataset=None, appliances=[])"
2012-01-01 00:00:00,0.0,0.0
2012-01-01 00:00:01,200.0,0.0
2012-01-01 00:00:02,0.0,1000.0
2012-01-01 00:00:03,200.0,1000.0
2012-01-01 00:00:04,0.0,0.0


In [27]:
pred.dtypes='float32'

AttributeError: can't set attribute